In [1]:
!git clone -b sdxl https://github.com/kohya-ss/sd-scripts
%cd /home/ec2-user/SageMaker/sd-scripts/
!sed -i 's/diffusers\[torch\]==0\.18\.2/diffusers[torch]==0.19.3/g; s/tensorboard==2\.10\.1/tensorboard==2.11.0/g' requirements.txt
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 --index-url https://download.pytorch.org/whl/cu118
!pip install -U -r requirements.txt
!pip install xformers==0.0.20 wandb prodigyopt

fatal: destination path 'sd-scripts' already exists and is not an empty directory.
/home/ec2-user/SageMaker/sd-scripts
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 458.9 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 55.7 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 30.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lit: filename=lit-15.0.7-py3-none-any.whl size=89988 sha256=a7804cca7f925ccba7374ac81c9f5deaa0590873152436e4ec7166853e247e4e
  Stored in directory: /home/ec2-user/.cache/pip/wheels/bf/62/0d/01218f13f6a8051e982a7ce31d12b7bfd725dc69bd227ae104
Successfully built lit
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1
    Uninstalling torch-2.0.1:
      Successfully uninstalled torch-2.0.1
  

In [2]:
# fix some warnings and bugs poping up from notebook

!sudo rm -rf /lib64/libstdc++.so.6 /lib64/libstdc++.so.6
!sudo ln -s /home/ec2-user/anaconda3/envs/python3/lib/libstdc++.so.6 /lib64/libstdc++.so.6

import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

## 2. Initialize training environment

In [3]:
from accelerate.utils import write_basic_config
write_basic_config()

PosixPath('/home/ec2-user/.cache/huggingface/accelerate/default_config.yaml')

In [5]:
%%writefile dataset.toml
[general]
enable_bucket = true
shuffle_caption = true
caption_extension = '.txt'
keep_tokens = 0

# DreamBooth caption based character datasets
[[datasets]]
resolution = 1024
# min_bucket_reso = 640
# max_bucket_reso = 1536
bucket_reso_steps = 32
batch_size = 2

  [[datasets.subsets]]
  image_dir = './images/sample'
  num_repeats = 10

Writing dataset.toml


In [6]:
%%writefile sample_prompts.txt
# prompt 1
wta, 1girl, looking_at_viewer, blue_hair, short_twintails, hair_ornament, blue_eyes, blush, smile, open_mouth, shirt, skirt, kneehighs, brown_footwear, standing, solo --n lowres, worst quality, ugly, extra limbs, deformed legs, disfigured legs, (disfigured), ((mutated hands, misshapen hands, mutated fingers, fused fingers):1.2) --w 1024 --h 1024 --d 3129467234 --l 7.5 --s 20

# # prompt 2
wta, 1girl, looking_at_viewer, blue_hair, short_twintails, hair_ornament, blue_eyes, blush, smile, open_mouth, shirt, skirt, kneehighs, brown_footwear, standing, solo --n lowres, worst quality, ugly, extra limbs, deformed legs, disfigured legs, (disfigured), ((mutated hands, misshapen hands, mutated fingers, fused fingers):1.2) --w 1024 --h 1024 --d 3129467235 --l 7.5 --s 20

Writing sample_prompts.txt


In [14]:
MODEL_NAME = "stabilityai/stable-diffusion-xl-base-1.0"
DATASET_CONFIG = "./dataset.toml"
TRAIN_DATA_DIR = "./images/sample"
IMAGES_OUTPTS = "./images/outputs"
LORA_WEIGHT = "./lora_weight"
!mkdir -p $TRAIN_DATA_DIR $IMAGES_OUTPTS $LORA_WEIGHT

In [15]:
!git clone https://github.com/terrificdm/dreambooth-stablediffusion-sagemaker-notebook
!cp -r dreambooth-stablediffusion-sagemaker-notebook/images/Wikipe-tan/* $TRAIN_DATA_DIR
!rm -rf dreambooth-stablediffusion-sagemaker-notebook

Cloning into 'dreambooth-stablediffusion-sagemaker-notebook'...
remote: Enumerating objects: 914, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 914 (delta 7), reused 14 (delta 4), pack-reused 895
Receiving objects: 100% (914/914), 124.22 MiB | 21.65 MiB/s, done.
Resolving deltas: 100% (236/236), done.


## 3. Train model

## LoRA Training with --Multi-gpu option

In [ ]:
!accelerate launch --multi_gpu sdxl_train_network.py \
   --pretrained_model_name_or_path=$MODEL_NAME \
   --dataset_config=$DATASET_CONFIG \
   --output_dir=$LORA_WEIGHT \
   --network_module="networks.lora" \
   --max_train_epochs=10 \
   --network_train_unet_only \
   --learning_rate=1.0 \
   --lr_scheduler="cosine_with_restarts" \
   --lr_scheduler_num_cycles=1 \
   --network_dim=64 \
   --network_alpha=32 \
   --output_name="lora_wta" \
   --save_every_n_epochs=2 \
   --mixed_precision="bf16" \
   --full_bf16 \
   --gradient_checkpointing \
   --max_token_length=225 \
   --save_model_as="safetensors" \
   --no_half_vae \
   --xformers \
   --optimizer_type="prodigy" \
   --min_snr_gamma=5 \
   --sample_every_n_epochs=2 \
   --sample_prompts="./sample_prompts.txt" \
   --sample_sampler="euler_a" \
   --logging_dir="./logs" \
   --log_with="all" \
   --log_tracker_name="lora_wta" \
   --wandb_api_key="245e51c77454f92bbb80ff882df76d02cfd7f4a5" # register a user via https://wandb.ai and get an API key 
   # --cache_text_encoder_outputs_to_disk \
   # --save_model_as="safetensors" \

prepare tokenizers
prepare tokenizers
prepare tokenizers
prepare tokenizers
update token length: 225
Loading dataset config from dataset.toml
update token length: 225
Loading dataset config from dataset.toml
update token length: 225
Loading dataset config from dataset.toml
update token length: 225
Loading dataset config from dataset.toml
prepare images.
found directory ./images/sample contains 12 image files
120 train images with repeating.
0 reg images.
no regularization images / 正則化画像が見つかりませんでした
[Dataset 0]
  batch_size: 2
  resolution: (1024, 1024)
  enable_bucket: True
  min_bucket_reso: 256
  max_bucket_reso: 1024
  bucket_reso_steps: 32
  bucket_no_upscale: False

  [Subset 0 of Dataset 0]
    image_dir: "./images/sample"
    image_count: 12
    num_repeats: 10
    shuffle_caption: True
    keep_tokens: 0
    caption_dropout_rate: 0.0
    caption_dropout_every_n_epoches: 0
    caption_tag_dropout_rate: 0.0
    color_aug: False
    flip_aug: False
    face_crop_aug_range: None
   

## 3. Use AA pic and caption to training model

In [4]:
%cd /home/ec2-user/SageMaker/sd-scripts/

/home/ec2-user/SageMaker/sd-scripts


In [14]:
%%writefile aa/dataset.toml
[general]
  enable_bucket = true
  shuffle_caption = true
  caption_extension = ".txt"
  keep_tokens = 0

[[datasets]]
  resolution = 1024
  batch_size = 1

  [[datasets.subsets]]
    image_dir = "./aa/images/sample"
    num_repeats = 20

Overwriting aa/dataset.toml


In [5]:
MODEL_NAME = "stabilityai/stable-diffusion-xl-base-1.0"
DATASET_CONFIG = "./aa/dataset.toml"
TRAIN_DATA_DIR = "./aa/images/sample"
IMAGES_OUTPTS = "./aa/images/outputs"
LORA_WEIGHT = "./aa/lora_weight"
!mkdir -p $TRAIN_DATA_DIR $IMAGES_OUTPTS $LORA_WEIGHT

In [6]:
!accelerate launch --multi_gpu sdxl_train_network.py \
   --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \
   --dataset_config=$DATASET_CONFIG \
   --output_dir=$LORA_WEIGHT \
   --network_module="networks.lora" \
   --max_train_epochs=10 \
   --network_train_unet_only \
   --learning_rate=4e-4 \
   --lr_scheduler="constant" \
   --lr_scheduler_num_cycles=1 \
   #--lr_scheduler_num_cycles=10 \
   --network_dim=128 \
   #--network_alpha=1 \
   --network_alpha=32 \
   --output_name="lora-aa-256" \
   --save_every_n_epochs=10 \
   --mixed_precision="fp16" \
   #--mixed_precision="bf16" \
   #--save_precision="bf16" \
   --gradient_checkpointing \
   --prior_loss_weight=1 \
   --max_token_length=225 \
   --save_model_as="safetensors" \
   --no_half_vae \
   --xformers \
   --optimizer_type="prodigy" \
   #--optimizer_type="Adafactor" \
   #--sample_every_n_epochs=10 \
   #--sample_prompts="./sample_prompts.txt" \
   #--sample_sampler="euler_a" \
   --logging_dir="./logs" \
   --log_with="all" \
   --wandb_api_key="245e51c77454f92bbb80ff882df76d02cfd7f4a5" \
   # --text_encoder_lr=4e-4 \
   # --unet_lr=4e-4 \
   # --cache_latents_to_disk \
   # --optimizer_args \
   # --max_data_loader_n_worker=0 \
   # --noise_offset=0.0 \
   --log_tracker_name="lora-aa-256" # register a user via https://wandb.ai and get an API key 
   # --cache_text_encoder_outputs_to_disk \
   # --save_model_as="safetensors" \

prepare tokenizersprepare tokenizersprepare tokenizersprepare tokenizers



Loading dataset config from aa/dataset.toml
Loading dataset config from aa/dataset.toml
Loading dataset config from aa/dataset.toml
Loading dataset config from aa/dataset.toml
prepare images.
found directory ./aa/images/sample contains 9 image files
180 train images with repeating.
0 reg images.
no regularization images / 正則化画像が見つかりませんでした
[Dataset 0]
  batch_size: 1
  resolution: (1024, 1024)
  enable_bucket: True
  min_bucket_reso: 256
  max_bucket_reso: 1024
  bucket_reso_steps: 64
  bucket_no_upscale: False

  [Subset 0 of Dataset 0]
    image_dir: "./aa/images/sample"
    image_count: 9
    num_repeats: 20
    shuffle_caption: True
    keep_tokens: 0
    caption_dropout_rate: 0.0
    caption_dropout_every_n_epoches: 0
    caption_tag_dropout_rate: 0.0
    color_aug: False
    flip_aug: False
    face_crop_aug_range: None
    random_crop: False
    token_warmup_min: 1,
    token_warmup_step: 0,
    is_reg: F

# Below are testing, not to run

## LoRA Training with single gpu

In [26]:
# !accelerate launch --multi_gpu --num_processes 4 sdxl_train_network.py \
#    --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \
#    --dataset_config=$DATASET_CONFIG \
#    --output_dir=$LORA_WEIGHT \
#    --network_module="networks.lora" \
#    --max_train_epochs=10 \
#    --network_train_unet_only \
#    --learning_rate=4e-4 \
#    --lr_scheduler="constant" \
#    --lr_scheduler_num_cycles=1 \
#    --network_dim=128 \
#    --network_alpha=32 \
#    --output_name="lora-aa" \
#    --save_every_n_epochs=10 \
#    --mixed_precision="fp16" \
#    --gradient_checkpointing \
#    --prior_loss_weight=1 \
#    --max_token_length=225 \
#    --save_model_as="safetensors" \
#    --no_half_vae \
#    --xformers \
#    --optimizer_type="prodigy" \
#    --sample_every_n_epochs=10 \
#    --sample_prompts="./sample_prompts.txt" \
#    --sample_sampler="euler_a" \
#    --logging_dir="./logs" \
#    --log_with="all" \
#    --wandb_api_key="245e51c77454f92bbb80ff882df76d02cfd7f4a5" \
#    --log_tracker_name="lora-aa" # register a user via https://wandb.ai and get an API key 
#    # --cache_text_encoder_outputs_to_disk \

prepare tokenizers
prepare tokenizers
prepare tokenizers
prepare tokenizers
update token length: 225
Loading dataset config from aa/dataset.toml
update token length: 225
Loading dataset config from aa/dataset.toml
update token length: 225
Loading dataset config from aa/dataset.toml
update token length: 225
Loading dataset config from aa/dataset.toml
prepare images.
found directory ./aa/images/sample contains 10 image files
neither caption file nor class tokens are found. use empty caption for ./aa/images/sample/IMG_1421.jpg / キャプションファイルもclass tokenも見つかりませんでした。空のキャプションを使用します: ./aa/images/sample/IMG_1421.jpg
neither caption file nor class tokens are found. use empty caption for ./aa/images/sample/IMG_1556.jpg / キャプションファイルもclass tokenも見つかりませんでした。空のキャプションを使用します: ./aa/images/sample/IMG_1556.jpg
neither caption file nor class tokens are found. use empty caption for ./aa/images/sample/IMG_1579.jpg / キャプションファイルもclass tokenも見つかりませんでした。空のキャプションを使用します: ./aa/images/sample/IMG_1579.jpg
neither capt